In [2]:
!pip install optuna


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 14.9 MB/s eta 0:00:00


In [11]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [24]:
import pandas as pd
import sqlite3
import joblib
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import MinMaxScaler
import optuna
from sklearn.metrics import mean_squared_error
import os

In [25]:
# Rutas
DB_PATH = "/content/drive/MyDrive/bd_movies_recommendation/data/db_movies.sqlite"
MODEL_PATH = "/content/drive/MyDrive/bd_movies_recommendation/models/best_knn_model.pkl"
COLUMNS_PATH = "/content/drive/MyDrive/bd_movies_recommendation/models/model_columns.pkl"

# Cargar datos
def load_data(db_path):
    print("Cargando datos...")
    conn = sqlite3.connect(db_path)
    df_ratings = pd.read_sql_query("SELECT userId, movieId, rating FROM ratings", conn)
    conn.close()
    return df_ratings

# Preprocesamiento de datos
def preprocess_data(df_ratings):
    print("Preprocesando los datos...")

    # Crear matriz usuario-película
    user_movie_matrix = df_ratings.pivot(index='userId', columns='movieId', values='rating').fillna(0)

    # Escalar los ratings entre 0 y 1
    scaler = MinMaxScaler()
    user_movie_matrix = pd.DataFrame(scaler.fit_transform(user_movie_matrix),
                                     columns=user_movie_matrix.columns,
                                     index=user_movie_matrix.index)

    # Guardar columnas para el despliegue
    columns = user_movie_matrix.columns.tolist()
    joblib.dump(columns, COLUMNS_PATH)
    print(f"Columnas guardadas en: {COLUMNS_PATH}")

    return user_movie_matrix


In [26]:
# Función objetivo para Optuna
def objective(trial, X):
    n_neighbors = trial.suggest_int('n_neighbors', 3, 20)
    metric = trial.suggest_categorical('metric', ['cosine', 'euclidean', 'manhattan'])

    # Entrenar el modelo KNN
    model = NearestNeighbors(n_neighbors=n_neighbors, metric=metric)
    model.fit(X)

    # Evaluación del modelo
    distances, indices = model.kneighbors(X)
    mse = mean_squared_error(distances, np.zeros_like(distances))
    return mse

In [27]:
# Optimización del modelo
def optimize_model(X):
    print("Optimizando el modelo con Optuna...")
    study = optuna.create_study(direction='minimize')
    study.optimize(lambda trial: objective(trial, X), n_trials=20)

    print(f"Mejores hiperparámetros: {study.best_params}")
    return study.best_params

In [28]:
# Entrenar y guardar el modelo
def train_and_save_model(X, best_params):
    print("Entrenando el modelo con los mejores parámetros...")
    model = NearestNeighbors(**best_params)
    model.fit(X)

    # Guardar el modelo
    joblib.dump(model, MODEL_PATH)
    print(f"Modelo guardado en: {MODEL_PATH}")

In [33]:
# Flujo principal
def main():
    # Cargar datos
    print("Cargando datos...")
    df_ratings = load_data(DB_PATH)

    # Preprocesamiento
    print("Preprocesando los datos...")
    X = preprocess_data(df_ratings)

    # Optimización del modelo
    print("Optimizando el modelo con Optuna...")
    best_params = optimize_model(X)

    # Entrenar y guardar modelo
    print("Entrenando el modelo con los mejores parámetros...")
    model = train_and_save_model(X, best_params)

    # Mostrar el modelo entrenado
    print("\n--- Modelo Entrenado ---")
    print(model)

if __name__ == "__main__":
    main()




Cargando datos...
Cargando datos...
Preprocesando los datos...
Preprocesando los datos...


[I 2025-05-11 20:46:13,778] A new study created in memory with name: no-name-2af9ffe8-bb0c-4198-a419-d8b3b7c2f7fd


Columnas guardadas en: /content/drive/MyDrive/bd_movies_recommendation/models/model_columns.pkl
Optimizando el modelo con Optuna...
Optimizando el modelo con Optuna...


[I 2025-05-11 20:46:19,772] Trial 0 finished with value: 45870.64050874514 and parameters: {'n_neighbors': 13, 'metric': 'manhattan'}. Best is trial 0 with value: 45870.64050874514.
[I 2025-05-11 20:46:20,114] Trial 1 finished with value: 89.35640202197943 and parameters: {'n_neighbors': 13, 'metric': 'euclidean'}. Best is trial 1 with value: 89.35640202197943.
[I 2025-05-11 20:46:20,458] Trial 2 finished with value: 88.5113282149001 and parameters: {'n_neighbors': 12, 'metric': 'euclidean'}. Best is trial 2 with value: 88.5113282149001.
[I 2025-05-11 20:46:20,796] Trial 3 finished with value: 69.97983605331173 and parameters: {'n_neighbors': 4, 'metric': 'euclidean'}. Best is trial 3 with value: 69.97983605331173.
[I 2025-05-11 20:46:21,417] Trial 4 finished with value: 0.47006114745085564 and parameters: {'n_neighbors': 18, 'metric': 'cosine'}. Best is trial 4 with value: 0.47006114745085564.
[I 2025-05-11 20:46:22,022] Trial 5 finished with value: 83.42447057743104 and parameters: {

Mejores hiperparámetros: {'n_neighbors': 9, 'metric': 'cosine'}
Entrenando el modelo con los mejores parámetros...

Entrenando el modelo con los mejores parámetros...
Modelo guardado en: /content/drive/MyDrive/bd_movies_recommendation/models/best_knn_model.pkl

--- Modelo Entrenado ---
NearestNeighbors(metric='cosine', n_neighbors=9)


In [34]:
# Ejemplo de cómo usar el modelo para hacer recomendaciones para un usuario
def get_movie_recommendations(user_id, model, df_pivot, df_movies, n_recommendations=5):
    """
    Genera recomendaciones de películas para un usuario específico.

    Args:
    - user_id (int): ID del usuario.
    - model (NearestNeighbors): Modelo KNN entrenado.
    - df_pivot (pd.DataFrame): Matriz usuario-película.
    - df_movies (pd.DataFrame): DataFrame con los títulos de las películas.
    - n_recommendations (int): Número de recomendaciones.

    Returns:
    - List[Tuple[str, float]]: Lista de recomendaciones con título y distancia.
    """
    # Verificar si el modelo está entrenado
    if model is None:
        print("El modelo no ha sido entrenado.")
        return []

    # Verificar si el usuario está en la matriz
    if user_id not in df_pivot.index:
        print(f"El usuario {user_id} no existe en la matriz de usuarios.")
        return []

    # Obtener las películas vistas por el usuario
    user_ratings = df_pivot.loc[user_id]

    # Filtrar películas no vistas (valor 0)
    unseen_movies = user_ratings[user_ratings == 0].index.tolist()

    # Si no hay películas no vistas, no hay recomendaciones
    if not unseen_movies:
        print(f"El usuario {user_id} ha visto todas las películas.")
        return []

    # Crear vector del usuario
    user_vector = user_ratings.values.reshape(1, -1)

    # Obtener distancias y recomendaciones
    distances, indices = model.kneighbors(user_vector, n_neighbors=n_recommendations)

    # Crear lista de recomendaciones
    recommendations = []
    for i, idx in enumerate(indices[0]):
        movie_id = df_pivot.columns[idx]
        movie_title = df_movies.loc[df_movies['movieId'] == movie_id, 'title'].values[0]
        recommendations.append((movie_title, distances[0][i]))

    return recommendations

# Flujo principal
def main():
    # Cargar datos
    df_ratings = load_data(DB_PATH)
    df_movies = pd.read_sql_query("SELECT movieId, title FROM movies", sqlite3.connect(DB_PATH))

    # Preprocesamiento
    df_pivot = preprocess_data(df_ratings)

    # Optimización del modelo
    best_params = optimize_model(df_pivot)

    # Entrenar y guardar modelo
    best_knn = train_and_save_model(df_pivot, best_params)

    # Verificar si el modelo fue entrenado correctamente
    if best_knn is None:
        print("Error: El modelo no fue entrenado correctamente.")
        return

    # Mostrar el modelo entrenado
    print("\n--- Modelo Entrenado ---")
    print(best_knn)

    # Recomendaciones para un usuario
    user_id = 1  # Puedes cambiar este ID para probar con otros usuarios
    recommendations = get_movie_recommendations(user_id, best_knn, df_pivot, df_movies)
    print(f"\nRecomendaciones para el usuario {user_id}:")
    if recommendations:
        for movie, distance in recommendations:
            print(f"Película recomendada: {movie}, Distancia: {distance:.2f}")
    else:
        print("No se generaron recomendaciones.")

if __name__ == "__main__":
    main()


Cargando datos...
Preprocesando los datos...


[I 2025-05-11 20:46:41,056] A new study created in memory with name: no-name-ba2dfe64-5edc-464a-9866-7b95c55cf3df


Columnas guardadas en: /content/drive/MyDrive/bd_movies_recommendation/models/model_columns.pkl
Optimizando el modelo con Optuna...


[I 2025-05-11 20:46:41,416] Trial 0 finished with value: 90.0993107955397 and parameters: {'n_neighbors': 14, 'metric': 'euclidean'}. Best is trial 0 with value: 90.0993107955397.
[I 2025-05-11 20:46:47,498] Trial 1 finished with value: 45870.64050874514 and parameters: {'n_neighbors': 13, 'metric': 'manhattan'}. Best is trial 0 with value: 90.0993107955397.
[I 2025-05-11 20:46:48,121] Trial 2 finished with value: 0.4658085097795673 and parameters: {'n_neighbors': 17, 'metric': 'cosine'}. Best is trial 2 with value: 0.4658085097795673.
[I 2025-05-11 20:46:49,015] Trial 3 finished with value: 0.4034026689604237 and parameters: {'n_neighbors': 8, 'metric': 'cosine'}. Best is trial 3 with value: 0.4034026689604237.
[I 2025-05-11 20:46:49,620] Trial 4 finished with value: 92.38166393188777 and parameters: {'n_neighbors': 18, 'metric': 'euclidean'}. Best is trial 3 with value: 0.4034026689604237.
[I 2025-05-11 20:46:50,441] Trial 5 finished with value: 0.3904279550548508 and parameters: {'n

Mejores hiperparámetros: {'n_neighbors': 3, 'metric': 'cosine'}

Entrenando el modelo con los mejores parámetros...
Modelo guardado en: /content/drive/MyDrive/bd_movies_recommendation/models/best_knn_model.pkl

--- Modelo Entrenado ---
NearestNeighbors(metric='cosine', n_neighbors=3)

Recomendaciones para el usuario 1:
Película recomendada: Toy Story (1995), Distancia: 0.00
Película recomendada: Ready to Wear (Pret-A-Porter) (1994), Distancia: 0.65
Película recomendada: Cobb (1994), Distancia: 0.66
Película recomendada: Don't Be a Menace to South Central While Drinking Your Juice in the Hood (1996), Distancia: 0.67
Película recomendada: Dead Presidents (1995), Distancia: 0.68
